## 範例目標

* 了解如何計算傳統電腦視覺特徵 hog
* 了解如何利用 opencv 調用 SVM

## 範例重點
* [產生 HOG 特徵的訓練資料](#產生-HOG-特徵的訓練資料)
* [用 histogram 特徵訓練 SVM 模型](#用-histogram-特徵訓練-SVM-模型)
* [用 HOG 特徵訓練 SVM 模型](#用-HOG-特徵訓練-SVM-模型)

In [1]:
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


In [5]:
x_train_histogram.shape

(50000, 48)

In [6]:
x_test_histogram.shape

(10000, 48)

#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [7]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

In [8]:
x_train_hog.shape

(50000, 64)

#### SVM model
* SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [該知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

#### 用 histogram 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [9]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

In [12]:
y_hist_test.shape

(10000, 1)

#### 用 HOG 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [10]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

In [13]:
y_hog_test.shape

(10000, 1)

## 作業目標
* 嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [15]:
y_test.shape

(10000, 1)

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_hog_test, y_test)

array([[234, 112,  37,  14,  33,  16,  19,  22, 293,  72],
       [135, 365,  58,  63,  50,  34,  83,  69, 211, 234],
       [225,  57, 280, 121, 234,  94, 182,  77, 109,  96],
       [ 17,  78,  25,  64,  26,  39,  27,  61,  30,  83],
       [ 18,   3,  10,  14,  15,  19,  11,  11,   7,   6],
       [ 40,   6, 193, 277, 179, 369, 174, 177,  10,   8],
       [ 21,  25,  34,  50,  20,  43, 100,  51,   5,  30],
       [141, 109, 269, 287, 375, 312, 281, 421, 147, 183],
       [ 86,  96,  12,   6,   7,   2,   4,   4, 122,  47],
       [ 83, 149,  82, 104,  61,  72, 119, 107,  66, 241]])

In [18]:
from sklearn.metrics import precision_score
print('histo',precision_score(y_test,y_hist_test,average='weighted'))
print('hog',precision_score(y_test,y_hog_test,average='weighted'))

histo 0.10151280835751897
hog 0.22450910670336188


In [20]:
from sklearn.metrics import accuracy_score
print('histo',accuracy_score(y_test,y_hist_test))
print('hog',accuracy_score(y_test,y_hog_test))

histo 0.1228
hog 0.2211
